In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler

import tensorflow as tf 
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer

## Bước 1: Đọc dữ liệu từ file excel/csv



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


## Bước 2: chuẩn hóa các cột dữ liệu thuộc dạng không có tình thứ tự (dữ liệu rời rạc) thành dạng One Hot Encoding

In [ ]:
geography = df.filter(['Geography'])
geography = pd.get_dummies(geography)
geography

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1
...,...,...,...
9995,1,0,0
9996,1,0,0
9997,1,0,0
9998,0,1,0


In [ ]:
gender = df['Gender'].replace(['Female','Male'],[0,1])
gender

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [ ]:
df = df.drop(columns=['Geography', 'Gender', 'Surname', 'RowNumber', 'CustomerId'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0
9996,516,35,10,57369.61,1,1,1,101699.77,0
9997,709,36,7,0.00,1,0,1,42085.58,1
9998,772,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df_new = df.join(geography)
df_new = df_new.join(gender)
df_new

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,1


In [ ]:
X = df_new.drop(columns=['Exited'])
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,0,1,0,1


In [ ]:
y = df_new['Exited']
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

## Bước 3: Chia dữ liệu ra làm 2 tập train và test 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

## Bước 4: Chuẩn hóa dữ liệu

In [ ]:
X_train = StandardScaler().fit_transform(X_train)
X_train

array([[ 0.35649971, -0.6557859 ,  0.34567966, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [-0.20389777,  0.29493847, -0.3483691 , ...,  1.72572313,
        -0.57638802,  0.91324755],
       [-0.96147213, -1.41636539, -0.69539349, ..., -0.57946723,
         1.73494238,  0.91324755],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ..., -0.57946723,
        -0.57638802, -1.09499335],
       [ 0.15932282,  0.3900109 ,  1.03972843, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [ 0.47065475,  1.15059039, -1.38944225, ...,  1.72572313,
        -0.57638802,  0.91324755]])

## Bước 5: Xây dựng mô hình Neural Network

In [ ]:
def build_model():
    model = Sequential()
    # model.add(Dense(6, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
    model.add(InputLayer(input_shape=(X_train.shape[1],)))
    model.add(Dense(6, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=1e-3),
        metrics=['accuracy']
    )
    return model

## Bước 6: Huấn luyện mạng với thuật toán Adam sử dụng hàm độ lỗi Binary Cross Entropy

In [ ]:
model = build_model()
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 7         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
epochs = 100
model.fit(X_train, y_train,
          epochs=epochs,
          verbose=1,
          shuffle=True)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5975 - accuracy: 0.7442
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4938 - accuracy: 0.7846
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.7925
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4447 - accuracy: 0.7966
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4358 - accuracy: 0.8036
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4298 - accuracy: 0.8102
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4255 - accuracy: 0.8131
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4218 - accuracy: 0.8175
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4184 - accuracy: 0.8195
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4153 - accura

## Bước 7: Test và đánh giá kết quả

In [ ]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 890us/step - loss: 893.0627 - accuracy: 0.7240


[893.0626831054688, 0.7239999771118164]

## Bước 8: Đánh giá độ chính xác bằng phương pháp KFold Cross Validation

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
history = []

In [ ]:
for train_idx, val_idx in cv.split(X, y):
    X_train = X.loc[train_idx]
    X_valid = X.loc[val_idx]
    y_train = y.loc[train_idx]
    y_test = y.loc[val_idx]

    model = build_model()
    model.fit(X_train, 
              y_train, 
                epochs=epochs,
                verbose=0,
                shuffle=True)
    history.append(model.evaluate(X_test, y_test))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


63/63 [==============================] - 0s 992us/step - loss: 1.7955 - accuracy: 0.7120


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


63/63 [==============================] - 0s 991us/step - loss: 4.5932 - accuracy: 0.6630


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


63/63 [==============================] - 0s 999us/step - loss: 0.5048 - accuracy: 0.7965


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


63/63 [==============================] - 0s 992us/step - loss: 5.9110 - accuracy: 0.7100


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


63/63 [==============================] - 0s 1ms/step - loss: 12.2226 - accuracy: 0.5785


In [ ]:
history

[[1.7954981327056885, 0.7120000123977661],
 [4.5931854248046875, 0.6629999876022339],
 [0.5048018097877502, 0.796500027179718],
 [5.911044597625732, 0.7099999785423279],
 [12.222590446472168, 0.578499972820282]]

In [ ]:
print(f"Mean accuracy: {np.mean([history[_][1] for _ in range(len(history))])}")

Mean accuracy: 0.6919999957084656
